# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
gmaps.configure(api_key=g_key)


In [3]:
city_df=pd.read_csv('city.csv')
city_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537 entries, 0 to 536
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   537 non-null    int64  
 1   Temperature  537 non-null    float64
 2   Latitude     537 non-null    float64
 3   Longitude    537 non-null    float64
 4   Humidity     537 non-null    int64  
 5   Wind Speed   537 non-null    float64
 6   Cloud        537 non-null    int64  
 7   Country      534 non-null    object 
 8   Date         537 non-null    int64  
 9   City         537 non-null    object 
dtypes: float64(4), int64(4), object(2)
memory usage: 42.1+ KB


In [4]:
Locations=city_df[["Latitude","Longitude"]]
Locations

,Latitude,Longitude
0,-42.7826,147.0587
1,-46.6000,168.3333
2,62.4560,-114.3525
3,35.7333,140.8333
4,-2.5333,112.9500
...,...,...
532,-34.2000,142.1500
533,9.3328,-69.1208
534,65.6118,72.6842
535,-16.6228,-72.7111


In [5]:
Weight=city_df["Humidity"]
Weight

0      70
1      70
2      48
3      80
4      74
       ..
532    63
533    92
534    77
535    89
536    85
Name: Humidity, Length: 537, dtype: int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
#Plot Heatmap
fig = gmaps.figure()

# # Create heat layer
heat_layer = gmaps.heatmap_layer(Locations, weights=Weight, 
                                 dissipating=False, max_intensity=20,
                               point_radius=1)


# # Add layer
fig.add_layer(heat_layer)

plt.savefig("images")

# Display figure
fig




Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
Weather_criteria_df=city_df[(city_df["Temperature"] <=70) & (city_df["Humidity"]<=80)] 
Weather_criteria_df=Weather_criteria_df.dropna()
Weather_criteria_df


,Unnamed: 0,Temperature,Latitude,Longitude,Humidity,Wind Speed,Cloud,Country,Date,City
0,0,52.65,-42.7826,147.0587,70,5.01,94,AU,1627433520,new norfolk
1,1,51.80,-46.6000,168.3333,70,11.18,44,NZ,1627433142,bluff
2,2,61.54,62.4560,-114.3525,48,1.99,40,CA,1627433623,yellowknife
8,8,46.56,-37.6167,-73.6500,80,22.10,0,CL,1627433116,lebu
11,11,64.58,50.6996,-127.4199,72,11.50,20,CA,1627433409,port hardy
...,...,...,...,...,...,...,...,...,...,...
521,521,66.63,37.0965,-4.3887,59,4.52,0,ES,1627433792,archidona
525,525,64.38,60.8156,-115.7999,55,4.61,75,CA,1627433793,hay river
526,526,49.23,65.4167,-52.9000,79,15.03,22,GL,1627433503,maniitsoq
532,532,60.76,-34.2000,142.1500,63,13.80,0,AU,1627433795,mildura


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
hotel_df=Weather_criteria_df[["City", "Country", "Latitude","Longitude"]]
hotel_df["Hotel Name"]=""
hotel_df

<ipython-input-8-e5edde5e9f04>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=""


,City,Country,Latitude,Longitude,Hotel Name
0,new norfolk,AU,-42.7826,147.0587,
1,bluff,NZ,-46.6000,168.3333,
2,yellowknife,CA,62.4560,-114.3525,
8,lebu,CL,-37.6167,-73.6500,
11,port hardy,CA,50.6996,-127.4199,
...,...,...,...,...,...
521,archidona,ES,37.0965,-4.3887,
525,hay river,CA,60.8156,-115.7999,
526,maniitsoq,GL,65.4167,-52.9000,
532,mildura,AU,-34.2000,142.1500,


In [9]:
Hotel_Name = []

for x in range(len(hotel_df)):
    Latitude = hotel_df.loc[index,'Latitude']
    Longitude = hotel_df.loc[index,'Longitude']

    params = {
        "location": f"{Latitude},{Longitude}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    Hotel_name= requests.get(base_url, params=params)
    Hotel_name = Hotel_name.json()
    try:
        Hotel_Name.append(jsn['results'][0]['name'])
    except:
        Hotel_Name.append("")
hotel_df["Hotel Name"] = Hotel_Name
hotel_df = hotel_df.dropna(how='any')
hotel_city_df.head()

NameError: name 'index' is not defined

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [15]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

plt.savefig("images")

fig





Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>